### TRAINING MODELS

In [1]:
# IMPORT EVERYTHING
#from Architectures.AE_ADJ import AE  # for some reason importing this makes it not work? (reconstructs everything around the brain somehow)
from Architectures.AE_ADJ_v2 import AE  # for some reason importing this makes it not work? (reconstructs everything around the brain somehow)

from Data.load import load, load_middle_slices
from Visualizations.latent_space_projections import plot_multiple_datasets
from Visualizations.plots import plot_middle_slices_in_range, plot_models_training_time, compare_models_loss_history, plot_images, compare_models_reconstruction
from Metrics.metrics_tf import MSE_loss, NMSE_loss, NRMSE_loss, SSIM_loss
from Metrics.metrics import NMSE, SSIM, NRMSE, MSE

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import layers, Model, activations, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from skimage.metrics import structural_similarity as ssim

import umap
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import glob
import tensorflow as tf
import time
import math
import nibabel as nib
import json
from tqdm import tqdm

c:\Users\kiran\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
groups = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    groups[group]=np.load(open(f"Data/D2-{group}-A.npy","br"))

Opening faster:   0%|          | 0/3 [00:00<?, ?it/s]

Opening faster: 100%|██████████| 3/3 [00:00<00:00, 54.92it/s]


In [5]:
"""
A2 = AE(name="A2", input_shape=groups["CN"].shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 16, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.Conv2D, 32, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
    ],
    decoder_layers=[
        (layers.Conv2D, 32, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 16, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
    ], VAE_model=False, optimizer="adam", loss_fn="mse").compile_model()
"""

public_repo = AE(name="public_repo", input_shape=groups["CN"].shape, latent_dim=128,
    encoder_layers=[
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
        (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization()
    ],
    decoder_layers=[
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same',}),
    ],
    VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model()

Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 128)


In [9]:
for m in [public_repo]: #[A2,A4,A5]
    m.train(groups["CN"], epochs=5, batch_size=128, verbose=True, 
    save_path=r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images", 
    save_interval=1000, patience=30)

Training public_repo on CPU
Input shape: (988, 80, 96)
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.0152 - val_loss: 0.0136
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 0.0144 - val_loss: 0.0128
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 0.0140 - val_loss: 0.0126
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 0.0139 - val_loss: 0.0126
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 0.0140 - val_loss: 0.0126
Models saved to C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9 as 'public_repo_autoencoder.keras', 'public_repo_encoder.keras', 'public_repo_decoder.keras'.
Loss history saved to C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9\history.json
Model details saved to C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9\configs.json
Training complete in 58.13s


In [6]:
#models_list = [AE.open(r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\A2\epoch_699")]

models_list = [AE.open(r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9")]

Models loaded successfully.


In [70]:

from sklearn.cluster import SpectralClustering
from scipy.stats import f_oneway
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.cluster import KMeans

def find_clusters(data, n_clusters=3):
    """
    Perform clustering on MRI latent space data using Spectral Clustering and calculate p-values.
    
    Parameters:
    data (numpy.ndarray): A matrix of shape (n_samples, m_features).
    n_clusters (int): Number of clusters to form.
    
    Returns:
    tuple: Cluster labels for each sample, p-values for samples.
    """
    
    # Perform Spectral Clustering
    spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
    labels = spectral.fit_predict(data)
    
    # Calculate p-values for each feature using ANOVA
    p_values_samples = []
    for i in range(data.shape[1]):
        feature_data = [data[labels == cluster, i] for cluster in range(n_clusters)]
        f_stat, p_value = f_oneway(*feature_data)
        p_values_samples.append(p_value)
    
    return labels, np.array(p_values_samples)

In [94]:
def Silhouette(data, labels):
    return silhouette_score(data, labels)

def Inertia(data, labels):
    # Inertia calculated using the labels from SpectralClustering
    inertia = 0
    n_clusters = len(set(labels))
    for i in range(n_clusters):
        cluster_data = data[labels == i]
        cluster_center = cluster_data.mean(axis=0)
        inertia += ((cluster_data - cluster_center) ** 2).sum()
    return inertia

def ARI(data, labels, true_labels):
    return adjusted_rand_score(true_labels, labels)



def find_clusters(data, n_clusters=3):
    """
    Perform clustering on MRI latent space data using Spectral Clustering and calculate p-values.
    
    Parameters:
    data (numpy.ndarray): A matrix of shape (n_samples, m_features).
    n_clusters (int): Number of clusters to form.
    
    Returns:
    tuple: Cluster labels for each sample, p-values for samples.
    """
    
    # Perform Spectral Clustering
    spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors', random_state=42)
    labels = spectral.fit_predict(data)
    
    # Calculate p-values for each feature using ANOVA
    p_values_samples = []
    for i in range(data.shape[1]):
        feature_data = [data[labels == cluster, i] for cluster in range(n_clusters)]
        f_stat, p_value = f_oneway(*feature_data)
        p_values_samples.append(p_value)
    
    return labels, np.array(p_values_samples)

def compare_models(models_list, groups, reconstruction_metrics, clustering_metrics):
    metric_results = []
    group_labels = {"CN": 0, "AD": 1, "MCI": 2}
    true_labels = []
    
    for label, image in groups.items():
        true_labels.extend([group_labels[label]] * image.shape[0])

    for i, model in enumerate(models_list):
        model_result = {"Model": model.name}

        for label, image in groups.items():
            latent = model.encode(image)
            recon = model.autoencode(image)

            for recon_metric in reconstruction_metrics:
                metric_name = f"{label}_{recon_metric.__name__}"
                metric_value = recon_metric(image, recon).numpy()
                model_result[metric_name] = metric_value
                print(f"Reconstruction metric computed: {metric_name}")

        combined_data = np.concatenate(list(groups.values()), axis=0)
        combined_data_scaled =  StandardScaler().fit_transform(combined_data.reshape(-1, combined_data.shape[-1])).reshape(combined_data.shape)
        
        latent = model.encode(combined_data_scaled)
        if model.VAE_model:
            latent = latent[2]


        # Run this shit several times to get the average result for UMAP and clustering, gosh
        n_runs = 20
        dim_red_latent_all_runs = []
        for i in range(n_runs):
            # Run UMAP with a different random state each time
            dim_red_latent = umap.UMAP(n_components=2).fit_transform(latent) #, random_state=i
            dim_red_latent_all_runs.append(dim_red_latent)

        # After collecting all the UMAP runs, now calculate the average for each metric
        for clustering_metric in clustering_metrics:
            metric_name = clustering_metric.__name__

            # Initialize lists to store metric values for each run
            metric_values = []

            for i in range(n_runs):
                cluster_labels, _ = find_clusters(dim_red_latent_all_runs[i])
                
                # Compute the clustering metric for each run
                if clustering_metric == ARI:
                    metric_value = clustering_metric(dim_red_latent_all_runs[i], cluster_labels, true_labels)
                else:
                    metric_value = clustering_metric(dim_red_latent_all_runs[i], cluster_labels)

                metric_values.append(metric_value)

            # Calculate the average metric across all runs
            average_metric_value = np.mean(metric_values)
            print(metric_values)

            # Store the averaged metric in the model results
            model_result[metric_name] = average_metric_value

        metric_results.append(model_result)

        # dim_red_latent = umap.UMAP(n_components=2).fit_transform(latent)
        # cluster_labels, _ = find_clusters(dim_red_latent)
        # for clustering_metric in clustering_metrics:
        #     metric_name = clustering_metric.__name__
        #     if clustering_metric == ARI:
        #         metric_value = clustering_metric(dim_red_latent, cluster_labels, true_labels)
        #     else:
        #         metric_value = clustering_metric(dim_red_latent, cluster_labels)
        #     model_result[metric_name] = metric_value
        # metric_results.append(model_result)

    return pd.DataFrame(metric_results)

results = []
for i in range(2):
    # To run this function
    # 1. add name to each model, model.name="something"
    # 2. add .VAE_model = True to your vae models, .VAE_model = false to others
    # 3. add in the encode(), decode() functions (you can copy from ae_adj_v2.py), or you can modify compare_models to your likings

    result = compare_models(
        models_list, groups,
        reconstruction_metrics=[SSIM_loss], 
        clustering_metrics=[Silhouette, Inertia, ARI]
    )
    results.append(result)
    print(display(result))

31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Reconstruction metric computed: CN_SSIM_loss
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
Reconstruction metric computed: MCI_SSIM_loss
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Reconstruction metric computed: AD_SSIM_loss
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step


c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-p

[0.09314602, 0.19893637, -0.028696671, -0.03430271, -0.034002222, 0.013561515, 0.049634404, 0.0056018136, -0.06966394, 0.063182905, 0.18837403, 0.016148472, 0.066128336, 0.0059129437, -0.15199271, 0.037259884, -0.13214289, 0.21147297, -0.03442305, -0.05858472]


c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-p

[170221.48866310716, 150278.89841650426, 191978.90947864205, 166710.15673828125, 187328.97339630127, 176907.576171875, 163550.2265625, 174034.2500038147, 204282.861328125, 197640.97509765625, 123667.10481681675, 185307.30406595021, 188660.0175794512, 156145.2047637105, 198661.31594634056, 184624.6421127133, 207858.8641526699, 130191.21846008301, 166933.88873291016, 196372.93377642334]


...

[116439.8623046875, 202330.12949385494, 178697.01138305664, 178665.65736805648, 177210.48123168945, 125750.85571289062, 191143.482421875, 128907.5101557225, 193917.09846305847, 173817.0771484375, 101646.763671875, 177851.28235131502, 164854.70098876953, 158584.06317138672, 111242.79565429688, 192409.5196647644, 188802.4230125621, 187860.49063110352, 150916.2921447754, 190331.26293945312]


c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\kiran\anaconda3\Lib\site-p

[0.21080495254795162, -0.005644557663730649, -0.0018616488193355504, 0.0295707145504631, 0.11072433841096263, 0.22187346253643692, 0.01634610875213476, 0.3352273714874713, -0.02295147186911349, 0.03709160509024372, 0.2035784229162516, -0.0017481312236319253, 0.07767967905982996, 0.10791577542586837, 0.19152807121218737, 0.052743418804109474, 0.04603226829331689, -0.008069657412389736, 0.10971610032880028, 0.0522171314354961]


,Model,CN_SSIM_loss,MCI_SSIM_loss,AD_SSIM_loss,Silhouette,Inertia,ARI
0,public_repo,0.451775,0.563819,0.562078,0.027232,164568.937996,0.088139


None
